In [112]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

In [113]:
pd.set_option('display.max_columns', None)
df_original = pd.read_excel("1 - Project Data.xlsx")
df_original.head()


,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,Competitor had better devices


In [114]:
X = df_original.drop(columns = ['Churn Label', 'Churn Value', 'Churn Reason'])
y = df_original['Churn Value']

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [116]:
def clean(df_original):
    # Mapping
    df_original['Gender'] = df_original.Gender.map({'Female':0, 'Male':1})
    df_original['Senior Citizen'] = df_original['Senior Citizen'].map({'No':0, 'Yes':1})
    df_original['Partner'] = df_original.Partner.map({'No':0, 'Yes':1})
    df_original['Dependents'] = df_original.Dependents.map({'No':0, 'Yes':1})
    df_original['Total Charges'] = pd.to_numeric(df_original['Total Charges'], errors='coerce')
    df_original['Total Charges'].fillna(0, inplace = True)
    df_original['Paperless Billing'] = df_original['Paperless Billing'].map({'No':0, 'Yes':1})   
    
    # Fill Nas
    df_original['Total Charges'].fillna(0, inplace=True)

    # OHE
    ohe_columns = ['Payment Method', 'Contract', 'Internet Service', 'Multiple Lines', 'Online Security',
     'Online Backup',
     'Device Protection',
     'Tech Support',
     'Streaming TV',
     'Streaming Movies']
    df_original = pd.get_dummies(df_original, columns = ohe_columns, drop_first = True, dtype = int)

    return df_original

In [117]:
X_train_fe = clean(X_train)
X_test_fe = clean(X_test)

C:\Users\rahul\AppData\Local\Temp\ipykernel_24712\95374104.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_original['Total Charges'].fillna(0, inplace = True)
C:\Users\rahul\AppData\Local\Temp\ipykernel_24712\95374104.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [118]:
def sanity_check(X, y):
    assert(len(X) == len(y)) , "Lengths dont match"
    assert(all(X.index == y.index)), "Indexs dont match"


In [119]:
sanity_check(X_train, y_train)

In [120]:
Feature_Columns = [#'CustomerID',
# 'Count',
 #'Country',
 #'State',
 #'City',
 #'Zip Code',
# 'Lat Long',
 #'Latitude',
#'Longitude',
 'Gender',
 'Senior Citizen',
 'Partner',
 'Dependents',
 #'Tenure Months',
 #'Phone Service',
# 'Multiple Lines',
# 'Internet Service',
# 'Online Security',
 #'Online Backup',
 #'Device Protection',
 #'Tech Support',
 #'Streaming TV',
 #'Streaming Movies',
 'Paperless Billing',
# 'Monthly Charges',
 #'Total Charges',
 #'Churn Label',
 #'Churn Value',
 #'Churn Reason',
'Payment Method_Credit card (automatic)',
 'Payment Method_Electronic check',
 'Payment Method_Mailed check',
 'Contract_One year',
 'Contract_Two year',
 'Internet Service_Fiber optic',
 'Internet Service_No',
 'Multiple Lines_No phone service',
 'Multiple Lines_Yes',
 'Online Security_No internet service',
 'Online Security_Yes',
 'Online Backup_No internet service',
 'Online Backup_Yes',
 'Device Protection_No internet service',
 'Device Protection_Yes',
 'Tech Support_No internet service',
 'Tech Support_Yes',
 'Streaming TV_No internet service',
 'Streaming TV_Yes',
 'Streaming Movies_No internet service',
 'Streaming Movies_Yes',
 'Lat_Scaled',
 'Lon_Scaled',
 'Monthly_Charges_Scaled',
 'Total_Charges_Scaled',
 'Tenure_Months_Scaled']

In [121]:
Feature_Columns

['Gender',
 'Senior Citizen',
 'Partner',
 'Dependents',
 'Paperless Billing',
 'Payment Method_Credit card (automatic)',
 'Payment Method_Electronic check',
 'Payment Method_Mailed check',
 'Contract_One year',
 'Contract_Two year',
 'Internet Service_Fiber optic',
 'Internet Service_No',
 'Multiple Lines_No phone service',
 'Multiple Lines_Yes',
 'Online Security_No internet service',
 'Online Security_Yes',
 'Online Backup_No internet service',
 'Online Backup_Yes',
 'Device Protection_No internet service',
 'Device Protection_Yes',
 'Tech Support_No internet service',
 'Tech Support_Yes',
 'Streaming TV_No internet service',
 'Streaming TV_Yes',
 'Streaming Movies_No internet service',
 'Streaming Movies_Yes',
 'Lat_Scaled',
 'Lon_Scaled',
 'Monthly_Charges_Scaled',
 'Total_Charges_Scaled',
 'Tenure_Months_Scaled']

In [122]:
scaler = MinMaxScaler()
scale_cols = ['Latitude', 'Longitude', 'Monthly Charges', 'Total Charges', 'Tenure Months']
scaler.fit(X_train_fe[scale_cols])
X_train_fe[['Lat_Scaled', 'Lon_Scaled', 'Monthly_Charges_Scaled', 'Total_Charges_Scaled', 'Tenure_Months_Scaled']] = scaler.transform(X_train_fe[scale_cols])
X_test_fe[['Lat_Scaled', 'Lon_Scaled', 'Monthly_Charges_Scaled', 'Total_Charges_Scaled','Tenure_Months_Scaled']] = scaler.transform(X_test_fe[scale_cols])

In [123]:
X_train_fe.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Paperless Billing,Monthly Charges,Total Charges,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check,Contract_One year,Contract_Two year,Internet Service_Fiber optic,Internet Service_No,Multiple Lines_No phone service,Multiple Lines_Yes,Online Security_No internet service,Online Security_Yes,Online Backup_No internet service,Online Backup_Yes,Device Protection_No internet service,Device Protection_Yes,Tech Support_No internet service,Tech Support_Yes,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes,Lat_Scaled,Lon_Scaled,Monthly_Charges_Scaled,Total_Charges_Scaled,Tenure_Months_Scaled
2870,9272-LSVYH,1,United States,California,Courtland,95615,"38.311609, -121.554034",38.311609,-121.554034,1,0,0,0,10,Yes,0,70.15,735.50,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.611907,0.271786,0.515695,0.084688,0.138889
5449,0931-MHTEM,1,United States,California,Big Bar,96010,"40.775272, -123.287414",40.775272,-123.287414,0,0,0,0,54,Yes,1,100.05,5299.65,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0.873823,0.100308,0.813652,0.610221,0.750000
6191,2122-YWVYA,1,United States,California,San Francisco,94116,"37.74441, -122.486764",37.744410,-122.486764,0,0,0,0,18,Yes,1,75.40,1380.40,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.551607,0.179514,0.568012,0.158944,0.250000
5275,4732-RRJZC,1,United States,California,Bridgeville,95526,"40.372532, -123.525626",40.372532,-123.525626,1,0,1,1,7,Yes,0,20.25,144.35,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0.831007,0.076742,0.018435,0.016621,0.097222
5962,8750-QWZAJ,1,United States,California,Orange,92867,"33.81859, -117.821288",33.818590,-117.821288,0,0,1,1,70,Yes,0,19.80,1378.75,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0.134246,0.641055,0.013951,0.158754,0.972222


In [124]:
X_train_fe = X_train_fe[Feature_Columns]
X_test_fe = X_test_fe[Feature_Columns]

In [126]:
X_test_fe.head()

,Gender,Senior Citizen,Partner,Dependents,Paperless Billing,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check,Contract_One year,Contract_Two year,Internet Service_Fiber optic,Internet Service_No,Multiple Lines_No phone service,Multiple Lines_Yes,Online Security_No internet service,Online Security_Yes,Online Backup_No internet service,Online Backup_Yes,Device Protection_No internet service,Device Protection_Yes,Tech Support_No internet service,Tech Support_Yes,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes,Lat_Scaled,Lon_Scaled,Monthly_Charges_Scaled,Total_Charges_Scaled,Tenure_Months_Scaled
1463,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0.141208,0.649868,0.205780,0.077129,0.250000
1517,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0.200867,0.633776,0.707524,0.130383,0.180556
5950,0,0,1,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0.134184,0.625908,0.664674,0.536334,0.763889
5206,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.516081,0.294905,0.567015,0.082955,0.138889
99,1,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0.104379,0.792433,0.015446,0.024727,0.152778
